Scraping Tutorial
https://www.youtube.com/watch?v=XjNm9bazxn8&index=5&list=WL

In [52]:
import requests
from bs4 import BeautifulSoup
import json
import datetime
import time
import random
import re

import pandas as pd
import glob
import csv

## TODO: 
- Dump all data into a single file instead of weekly files.
- Further cleanup of data types being pushed into the dict.

In [ ]:
def the_numbers_spider(max_pages):
    ''' Crawler starts from a hardcoded start date, scrapes the weekly data into a json file, 
    then increments by a week and repeats until the max page is reached.
    
    Args:
        max_pages (int): The maximum number of pages one wants to crawl.
    '''
    
    # Page counter.
    page = 1
    
#     # Dict to hold weekly data.
#     weeklyData = {}
    
    # Declare starting dates. This will be changed to the first weekly report back in 1977. First date was 5/27/1977.
    date = datetime.date(2017, 4, 28) # This creates a datetime object that is not sliceable like a string is.
    dateString = str(date)
    
    while page <= max_pages:
        url = "http://www.the-numbers.com/box-office-chart/weekly/" + dateString[0:4] + "/" + dateString[5:7] + "/" + dateString[8:10]

        # Take a break before requesting the URL to avoid blacklisting (in seconds).
        time.sleep(random.randint(10, 30))
    
        source_code = requests.get(url)
        plain_text = source_code.text
        soup = BeautifulSoup(plain_text, 'lxml')
        
        # Store all the table rows in the second table as a BS4 object.
        tableRows = soup.find_all('table')[1].find_all('tr')
        
        # Dict to hold weekly data.
        weeklyData = {}
        
        # Save the data into the dictionary.
        # Skips first row which holds header.
        for row in tableRows[1:]:
            title = row.a.get_text()
            weeklyData[title] = {}
            weeklyData[title]['weekEnded'] = dateString
            weeklyData[title]['currentRank'] = row.td.get_text()
            weeklyData[title]['prevRank'] = row.select('td')[1].get_text(strip=True)[1:-1] # Ignores the parens
            try:
                weeklyData[title]['distributor'] = row.select('a')[1].get_text(strip=True)
            except:
                pass
            weeklyData[title]['gross'] = int(row.select('td')[4].get_text(strip=True)[1:].replace(",", "")) # Finds the string for gross rev, strips lead/trailing spaces, replaces the commas with nothing, and ignores the first char which is a dollar sign. Whew.
            weeklyData[title]['change'] = row.select('td')[5].get_text(strip=True)
            weeklyData[title]['theaters'] = int(row.select('td')[6].get_text(strip=True).replace(",", ""))
            try:
                weeklyData[title]['perThtr'] = int(row.select('td')[7].get_text(strip=True)[1:].replace(",", ""))
            except:
                pass
            weeklyData[title]['totalGross'] = int(row.select('td')[8].get_text(strip=True)[1:].replace(",", ""))
            weeklyData[title]['grossDays'] = int(row.select('td')[9].get_text(strip=True).replace(",", ""))
            
#         # Debug print.
#         print(weeklyData)

        # Dump the dict into a json file.
        # Will think about writing it all to one giant file.
        with open('the-numbers_weekly ' + dateString + ".json", mode='wt', encoding='utf-8') as file:
            json.dump(weeklyData, file)
            
        # Status prints
        print(dateString)
        
        # Increment operations
        date += datetime.timedelta(days=7)
        dateString = str(date)            
        page += 1
        
    print("done!")

In [ ]:
the_numbers_spider(27)

In [ ]:
import shutil

def jsonConsolidator():
    '''Consolidate all the json scrape files while adding in the report date.
    '''
    # Find list of json files
    fileList = (glob.glob(r"C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\*.json"))
    


    for file in fileList:
        # Regex search for the date in file name
        date = (re.search(r'\d{4}-\d{2}-\d{2}', file)).group(0) 
        
        with open(file) as data_file:    
            data = json.load(data_file)
            
#             print("original data:")
#             print(list(data.keys()))
    
            keys = list(data.keys())

            for key in keys:
                data[key]['weekOf'] = date
        # #         print(key, data[key])

        #     print(data)


    # Concat ALL files
    with open('output_file.json','wb') as wfd:
        for f in fileList:
            with open(f,'rb') as fd:
                shutil.copyfileobj(fd, wfd, 1024*1024*10)
    
    print("Done!")
    

    
#     iter(data).next()['weekOf'] = newVar
    
#     print("updated data:")
#     print(data)

    # Open a new json file or csv as the data file
    # Point to directory and all the files
    # Loop
        # Store the date from file name into variable
            # Regex for pulling date: \d{4}-\d{2}-\d{2}
        # Copy data from json to data file, appending the report date to each line
    # Close the data file

In [ ]:
# Concat ALL files
# Find list of json files
fileList = (glob.glob(r"C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\*.json"))
with open('output_file.json','wb') as wfd:
    for f in fileList:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd, 1024*1024*10)

print("Done!")

In [ ]:
jsonConsolidator()

In [4]:
newData = pd.read_json(r"C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\output_file.json")

In [ ]:
newData

In [5]:
newData = newData.transpose()

In [6]:
newData

,change,currentRank,distributor,gross,grossDays,perThtr,prevRank,theaters,totalGross,weekEnded
"!Women, Art, Revolution",,113,Zeitgeist,775,24,775,-,1,52681,2011-11-04
$9.99,,85,Regent Releasing,544,31,544,-,1,52107,2010-01-15
$upercapitalist,-23%,91,NaN,1657,3,1657,87,1,15919,2012-08-24
'Luv Shuv Tey Chicken Khurana,-75%,55,UTV Communications,28603,2,753,38,38,143900,2012-11-09
'Tis Autumn: The Search for…,,108,Outsider,1476,1,1476,e,1,1476,2007-12-07
(500) Days of Summer,-53%,81,Fox Searchlight,10694,19,535,65,20,32391374,2009-11-20
(Untitled),-59%,81,NaN,6181,7,1236,66,5,230600,2009-12-04
*batteries not included,-36%,6,Universal,5625209,3,4115,5,1367,20287929,1988-01-01
...And They Lived Happily E…,-62%,116,Kino International,1157,27,289,109,4,252341,2005-10-07
1 + 1 = 11,,69,Eros Entertainment,36258,3,2014,-,18,132498,2003-04-11


In [65]:
masterList = []

# Find list of json files
fileList = (glob.glob(r"C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\*.json"))

with open('output_file.json','wb') as wfd:
    for f in fileList:
        with open(f,'rt') as fd:
            print(f)
            data = json.load(fd)

# with open('the-numbers_weekly 2010-01-15.json') as json_file:  
#     data = json.load(json_file)

            for key in data.keys():

                rowList = []
                rowList.append(key)

                try:
                    rowList.append(data[key]['change'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['currentRank'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['distributor'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['gross'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['grossDays'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['perThtr'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['prevRank'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['theaters'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['totalGross'])
                except:
                    rowList.append('')
                try:
                    rowList.append(data[key]['weekEnded'])
                except:
                    rowList.append('')

                masterList.append(rowList)
    
with open("output.csv", "w", encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(masterList)

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-05-27.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-06-03.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-06-10.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-06-17.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-06-24.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-07-01.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-07-08.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1977-07-15.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-08-03.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-08-10.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-08-17.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-08-24.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-08-31.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-09-07.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-09-14.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1984-09-21.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-02-09.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-02-16.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-02-23.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-03-02.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-03-09.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-03-16.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-03-23.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1990-03-30.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-04-07.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-04-14.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-04-21.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-04-28.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-05-05.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-05-12.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-05-19.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1995-05-26.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-02-26.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-03-05.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-03-12.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-03-19.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-03-26.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-04-02.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-04-09.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 1999-04-16.json
C:\Users

...

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-02-02.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-02-09.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-02-16.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-02-23.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-03-02.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-03-09.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-03-16.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2007-03-23.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-10-09.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-10-16.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-10-23.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-10-30.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-11-06.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-11-13.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-11-20.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2009-11-27.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-07-06.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-07-13.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-07-20.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-07-27.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-08-03.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-08-10.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-08-17.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2012-08-24.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-01-24.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-01-31.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-02-07.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-02-14.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-02-21.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-02-28.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-03-07.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2014-03-14.json
C:\Users

C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-10-14.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-10-21.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-10-28.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-11-04.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-11-11.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-11-18.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-11-25.json
C:\Users\lauaa\Documents\Cal MIDS\2017 DATASCI W209\Final\w209-movies\beautifulSoup_test\the-numbers_weekly 2016-12-02.json
C:\Users